In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np

torch.manual_seed(2019)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [4]:
train_data = torchvision.datasets.CIFAR100('./data/', train=True, transform= trans, download=True)
test_data = torchvision.datasets.CIFAR100('./data/', train=False, transform= trans, download=True)

0it [00:00, ?it/s]

169009152it [00:06, 27787644.58it/s]                               


Extracting ./data/cifar-100-python.tar.gz to ./data/
Files already downloaded and verified


In [0]:
batch_size = 64

conv_channel_1 = 128
conv_channel_2 = 256
conv_channel_3 = 256
conv_channel_4 = 512
conv_channel_5 = 512

hidden1_size = 1024
out_size = 100
epoch = 50

In [0]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=True)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [0]:
loss = nn.CrossEntropyLoss()

In [0]:
import torch.optim as optim
from torch.autograd import Variable

def fit(epoch, data_loader, model, opt, phase):
    if phase=='train':
        volatile = False
        model.train()
    else:
        volatile = True
        model.eval()       
        
    learning_loss = 0.
    learning_correct = 0
    
    for i, (inputs, labels) in enumerate(data_loader):
        if phase == 'train':
            opt.zero_grad()
            
        inputs = Variable(inputs, volatile).cuda()
        labels = Variable(labels).cuda()
        outputs = model(inputs)
        
        loss_value = loss(outputs, labels)
        learning_loss += loss_value.item() * inputs.shape[0]
        
        if phase == 'train':
            loss_value.backward()
            opt.step()
        
        _, preds = torch.max(outputs.data, 1)
        learning_correct += (preds == labels).sum().item()
        
    epoch_loss = learning_loss /len(data_loader.dataset)
    epoch_correct = learning_correct / len(data_loader.dataset) * 100.
    
    print(f'epoch = {epoch} -> {phase:{5}} / loss = {epoch_loss:{8}.{3}}, correct = {epoch_correct:{8}.{4}}%')

In [0]:
class commonLinear(nn.Module):
    def __init__(self, nin):
        super(commonLinear, self).__init__()
        self.fc=nn.Sequential(
            nn.Linear(nin, hidden1_size),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden1_size, out_size),
            nn.ReLU()
        )
        
    def forward(self,x):
        x=self.fc(x)
        return F.log_softmax(x, dim=1)

In [0]:
class normalCNN(nn.Module):
    def __init__(self):
        super(normalCNN,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_2, conv_channel_3, 3, padding=1),
            nn.BatchNorm2d(conv_channel_3),
            nn.ReLU(),
            nn.Conv2d(conv_channel_3, conv_channel_4, 3, padding=1),
            nn.BatchNorm2d(conv_channel_4),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_4, conv_channel_5, 3, padding=1),
            nn.BatchNorm2d(conv_channel_5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [0]:
cnn1 = normalCNN().cuda()
#opt1 = optim.Adadelta(cnn1.parameters(), rho=0.95)
opt1 = optim.SGD(cnn1.parameters(), lr=0.01, momentum=0.9)
print(cnn1)

normalCNN(
  (conv): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_r

In [0]:
for ep in range(epoch):
    t= time.time()
    fit(ep, train_loader, cnn1, opt1, 'train')
    fit(ep, test_loader, cnn1, opt1, 'eval')
    print('%.4f'%(time.time()-t))
    print()

epoch = 0 -> train / loss =     3.81, correct =    12.72%
epoch = 0 -> eval  / loss =     3.22, correct =    22.36%
60.9932

epoch = 1 -> train / loss =     2.95, correct =    26.18%
epoch = 1 -> eval  / loss =     2.52, correct =    34.62%
60.7587

epoch = 2 -> train / loss =     2.47, correct =    35.39%
epoch = 2 -> eval  / loss =      2.2, correct =    41.43%
60.6141

epoch = 3 -> train / loss =     2.14, correct =    42.56%
epoch = 3 -> eval  / loss =     2.12, correct =    43.65%
60.5708

epoch = 4 -> train / loss =     1.87, correct =    48.51%
epoch = 4 -> eval  / loss =      1.8, correct =    51.11%
60.5755

epoch = 5 -> train / loss =     1.66, correct =    53.72%
epoch = 5 -> eval  / loss =     1.67, correct =    53.77%
60.5526

epoch = 6 -> train / loss =     1.46, correct =    58.28%
epoch = 6 -> eval  / loss =     1.69, correct =    54.24%
60.5279

epoch = 7 -> train / loss =     1.29, correct =    62.32%
epoch = 7 -> eval  / loss =     1.66, correct =    55.26%
60.4979



KeyboardInterrupt: ignored

In [0]:
class depthwiseCNN(nn.Module):
    def __init__(self,nin,nout):
        super(depthwiseCNN,self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(nin,nin,3,groups=nin, padding=1),
            nn.BatchNorm2d(nin),
            nn.ReLU(),
            nn.Conv2d(nin,nout,1),
            nn.BatchNorm2d(nout),
            nn.ReLU()
        )

    def forward(self,x):
        return self.model(x)
        
class separableCNN(nn.Module):
    def __init__(self):
        super(separableCNN,self).__init__()
        self.conv = nn.Sequential(
            depthwiseCNN(3,conv_channel_1),
            depthwiseCNN(conv_channel_1,conv_channel_2),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_2,conv_channel_3),
            depthwiseCNN(conv_channel_3,conv_channel_4),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_4,conv_channel_5),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [0]:
cnn2 = separableCNN().cuda()
#opt2 = optim.Adadelta(cnn2.parameters(), rho=0.95)
opt2 = optim.SGD(cnn2.parameters(), lr=0.01, momentum=0.9)
print(cnn2)

separableCNN(
  (conv): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): depthwiseCNN(
      (model): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): depthwiseCNN(
      (model): Sequential(
        (0): Con

In [0]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn2, opt2, 'train')
    fit(ep, test_loader, cnn2, opt2, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =     3.75, correct =    13.73%
epoch = 0 -> eval  / loss =     2.95, correct =    25.72%
27.8952

epoch = 1 -> train / loss =     2.84, correct =     28.1%
epoch = 1 -> eval  / loss =     2.44, correct =    36.87%
27.8950

epoch = 2 -> train / loss =      2.4, correct =    37.34%
epoch = 2 -> eval  / loss =     2.17, correct =    43.15%
27.7103

epoch = 3 -> train / loss =     2.09, correct =    43.72%
epoch = 3 -> eval  / loss =     1.89, correct =    48.59%
28.1700

epoch = 4 -> train / loss =     1.84, correct =    49.12%
epoch = 4 -> eval  / loss =     1.72, correct =     52.4%
28.5500

epoch = 5 -> train / loss =     1.62, correct =     54.3%
epoch = 5 -> eval  / loss =     1.71, correct =    53.61%
28.2421

epoch = 6 -> train / loss =     1.43, correct =    58.74%
epoch = 6 -> eval  / loss =      1.7, correct =    54.07%
28.1134

epoch = 7 -> train / loss =     1.25, correct =    63.01%
epoch = 7 -> eval  / loss =     1.57, correct =    56.16%
28.1180



KeyboardInterrupt: ignored

In [0]:
class inception(nn.Module):
    def __init__(self, nin, nout):
        super(inception,self).__init__()
        self.model1x1 = nn.Sequential(
            nn.Conv2d(nin, int(nout/4), 1),
            nn.BatchNorm2d(int(nout/4))
        )
        self.model3x3 = nn.Sequential(
            nn.Conv2d(nin,int(nout/8),1),
            nn.BatchNorm2d(int(nout/8)),
            nn.Conv2d(int(nout/8),int(nout/4),3, padding=1),
            nn.BatchNorm2d(int(nout/4))
        )
        self.model5x5 = nn.Sequential(
            nn.Conv2d(nin,int(nout/16),1),
            nn.BatchNorm2d(int(nout/16)),
            nn.Conv2d(int(nout/16),int(nout/8),3,padding=1),
            nn.BatchNorm2d(int(nout/8)),
            nn.Conv2d(int(nout/8),int(nout/4),3,padding=1),
            nn.BatchNorm2d(int(nout/4))
        )
        self.modelmax = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(nin,int(nout/4),1),
            nn.BatchNorm2d(int(nout/4))
        )
        
    def forward(self, x):
        part1x1=self.model1x1(x)
        part3x3=self.model3x3(x)
        part5x5=self.model5x5(x)
        partmax=self.modelmax(x)
        outputs= [part1x1, part3x3, part5x5, partmax]
        return torch.cat(outputs,1)
    
class inceptionCNN(nn.Module):
    def __init__(self):
        super(inceptionCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            inception(conv_channel_2, conv_channel_3),
            inception(conv_channel_3, conv_channel_4),
            nn.MaxPool2d(2,2),
            inception(conv_channel_4, conv_channel_5),
            nn.AvgPool2d(2,2)
        )
        
        self.fc=commonLinear(conv_channel_5*4*4)
    
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1, conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [13]:
cnn3 = inceptionCNN().cuda()
#opt3 = optim.Adadelta(cnn3.parameters(), rho=0.95)
opt3 = optim.SGD(cnn3.parameters(), lr=0.01, momentum=0.9)
print(cnn3)

inceptionCNN(
  (conv): Sequential(
    (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): inception(
      (model1x1): Sequential(
        (0): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (model3x3): Sequential(
        (0): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): BatchNorm2d(64, eps=1e

In [0]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn3, opt3, 'train')
    fit(ep, test_loader, cnn3, opt3, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =     3.29, correct =    22.84%
epoch = 0 -> eval  / loss =     2.49, correct =    37.44%
114.3417

epoch = 1 -> train / loss =     2.36, correct =    39.75%
epoch = 1 -> eval  / loss =     2.13, correct =    44.02%
114.3092

epoch = 2 -> train / loss =     1.95, correct =    48.37%
epoch = 2 -> eval  / loss =     2.11, correct =    46.41%
114.3983

epoch = 3 -> train / loss =     1.65, correct =    54.91%
epoch = 3 -> eval  / loss =     1.87, correct =    50.62%
114.3929

epoch = 4 -> train / loss =     1.41, correct =    60.79%
epoch = 4 -> eval  / loss =     1.83, correct =    51.61%
114.3675

epoch = 5 -> train / loss =      1.2, correct =    65.52%
epoch = 5 -> eval  / loss =     1.69, correct =    55.44%
114.5998

epoch = 6 -> train / loss =     1.03, correct =     69.9%
epoch = 6 -> eval  / loss =     2.08, correct =     50.6%
114.4328

epoch = 7 -> train / loss =    0.873, correct =     74.0%
epoch = 7 -> eval  / loss =     1.83, correct =    54.46%
11